In [1]:
import pandas as pd
import numpy as np
import pymysql
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import randomized_svd 


# Ⅰ. 엘페이이용고객과 상품구매정보고객간 유사고객 찾기

## 가. MySQL에서 데이터 가져오기
### 1. 엘페이이용 고객 중 유통사에서 상품을 구매한 고객들의 이용횟수

In [2]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select cust ,cop_c, count(*) as 이용횟수 from 엘페이이용_view where cop_c like 'A%' group by cust, cop_c ORDER BY cust, cop_c;")

result = cur.fetchall()

conn.commit()
conn.close()

df_lpay = pd.DataFrame(result)

In [3]:
df_lpay.columns = ['cust','제휴사', '수량']

In [4]:
df_lpay

,cust,제휴사,수량
0,M000136117,A01,1
1,M000136117,A06,2
2,M000419293,A01,15
3,M000419293,A02,1
4,M000494848,A01,5
...,...,...,...
16112,M999770689,A06,2
16113,M999962961,A01,6
16114,M999962961,A02,1
16115,M999962961,A03,1



## 2. 상품구매정보 테이블 고객들의 상품 유통사별 이용횟수

In [5]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select cust ,cop_c, count(*) as 이용횟수 from 상품구매정보_view group by cust, cop_c order by cust, cop_c;")

result = cur.fetchall()

conn.commit()
conn.close()

df_product = pd.DataFrame(result)

In [6]:
df_product.columns = ['cust','제휴사', '수량']

In [7]:
df_product

,cust,제휴사,수량
0,M000034966,A01,4
1,M000034966,A02,44
2,M000034966,A04,3
3,M000136117,A01,89
4,M000136117,A02,10
...,...,...,...
67997,M999962961,A01,42
67998,M999962961,A02,136
67999,M999962961,A03,148
68000,M999962961,A05,1


## 나. 피벗테이블로 변환
### - 엘페이이용고객
### - 상품구매정보고객

In [8]:
df_lpay_pivot=df_lpay.pivot('cust','제휴사','수량')
df_lpay_pivot=df_lpay_pivot.fillna(0)

In [9]:
df_lpay_pivot.shape

(7915, 6)

In [10]:
df_lpay_pivot

제휴사,A01,A02,A03,A04,A05,A06
cust,,,,,,
M000136117,1.0,0.0,0.0,0.0,0.0,2.0
M000419293,15.0,1.0,0.0,0.0,0.0,0.0
M000494848,5.0,0.0,0.0,0.0,0.0,40.0
M000557840,0.0,38.0,0.0,0.0,0.0,0.0
M000871427,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
M999105944,0.0,0.0,0.0,0.0,0.0,1.0
M999268192,0.0,0.0,0.0,0.0,1.0,1.0
M999599111,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
df_product_pivot=df_product.pivot('cust','제휴사','수량')
df_product_pivot=df_product_pivot.fillna(0)

In [12]:
df_product_pivot.index

Index(['M000034966', 'M000136117', 'M000201112', 'M000225114', 'M000261625',
       'M000350564', 'M000419293', 'M000494848', 'M000504230', 'M000508243',
       ...
       'M999340261', 'M999441814', 'M999468256', 'M999492154', 'M999515910',
       'M999599111', 'M999673157', 'M999770689', 'M999849895', 'M999962961'],
      dtype='object', name='cust', length=26917)

In [13]:
df_product_pivot

제휴사,A01,A02,A03,A04,A05,A06
cust,,,,,,
M000034966,4.0,44.0,0.0,3.0,0.0,0.0
M000136117,89.0,10.0,0.0,0.0,0.0,10.0
M000201112,0.0,0.0,0.0,21.0,0.0,0.0
M000225114,156.0,0.0,3.0,0.0,0.0,0.0
M000261625,67.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
M999599111,1.0,28.0,0.0,4.0,0.0,1.0
M999673157,12.0,0.0,0.0,12.0,0.0,0.0
M999770689,22.0,120.0,24.0,149.0,0.0,0.0


## 다. 엘페이이용_유통사 고객과 상품구매정보 테이블의 고객들 간의 유사도 측정
### 1. 코사인 함수 사용

In [14]:
df_lpay_pivot=df_lpay.pivot('cust','제휴사','수량')    
df_lpay_pivot=df_lpay_pivot.fillna(0)      # 결측치 처리

df_product_pivot=df_product.pivot('cust','제휴사','수량')
df_product_pivot=df_product_pivot.fillna(0)      # 결측치 처리

cos = cosine_similarity(df_lpay_pivot,df_product_pivot)


In [15]:
# index = 상품 테이블의 고객
# columns = lpay 테이블의 고객
df1 = pd.DataFrame(cos, index=(df_lpay_pivot.index), columns=(df_product_pivot.index))
df1

cust,M000034966,M000136117,M000201112,M000225114,M000261625,M000350564,M000419293,M000494848,M000504230,M000508243,...,M999340261,M999441814,M999468256,M999492154,M999515910,M999599111,M999673157,M999770689,M999849895,M999962961
cust,,,,,,,,,,,,,,,,,,,,,
M000136117,0.040396,0.540925,0.000000,0.447131,0.447214,0.447214,0.442385,0.367190,0.009928,0.416015,...,0.000000,0.415227,0.019013,0.447214,0.022969,0.047375,0.316228,0.050698,0.216842,0.108875
M000419293,0.156221,0.992804,0.000000,0.997601,0.997785,0.997785,0.996761,0.777693,0.022149,0.952487,...,0.000000,0.926420,0.108879,0.997785,0.117678,0.101001,0.705541,0.154245,0.541586,0.248094
M000494848,0.011204,0.232608,0.000000,0.124012,0.124035,0.124035,0.122696,0.134000,0.002753,0.115382,...,0.000000,0.115163,0.005273,0.124035,0.006371,0.039418,0.087706,0.014061,0.060141,0.044691
M000557840,0.993605,0.110967,0.000000,0.000000,0.000000,0.000000,0.146549,0.671777,0.000000,0.365451,...,0.000000,0.000000,0.999096,0.000000,0.998680,0.988714,0.000000,0.618351,0.868731,0.662187
M000871427,0.090328,0.987609,0.000000,0.999815,1.000000,1.000000,0.989203,0.734634,0.022199,0.930238,...,0.000000,0.928477,0.042515,1.000000,0.051361,0.035311,0.707107,0.113364,0.484873,0.204499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999105944,0.000000,0.110967,0.000000,0.000000,0.000000,0.000000,0.000000,0.043214,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.035311,0.000000,0.000000,0.000000,0.019476
M999268192,0.000000,0.078466,0.000000,0.000000,0.000000,0.000000,0.000000,0.030557,0.007848,0.023492,...,0.707107,0.000000,0.000000,0.000000,0.000000,0.024969,0.000000,0.000000,0.000000,0.017215
M999599111,0.000000,0.110967,0.000000,0.000000,0.000000,0.000000,0.000000,0.043214,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.035311,0.000000,0.000000,0.000000,0.019476


### 2. 함수 생성-1
#### - 유사도가 가장 높은 고객 추출

In [16]:
# cust_id : 상품을 추천할 엘페이 고객번호
def most_sim(cust_id):
    
    return df1.loc[cust_id].sort_values(ascending=False).index[0]

In [17]:
most_sim('M999770689')

'M785544658'

# Ⅱ. 상품구매정보 고객간 유사한 고객 찾기

## 가. MySQL 에서 데이터 가져오기
### 1. 고객별 상품별 구매횟수

In [18]:
# 고객별 구매수량 및 금액 가져오기

conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select * from 구매정보_소분류;")

result = cur.fetchall()

conn.commit()
conn.close()

df = pd.DataFrame(result)

In [19]:
df

,0,1,2
0,M000034966,가공조미료,3
1,M000034966,간장,1
2,M000034966,감자스낵,2
3,M000034966,과일음료,1
4,M000034966,국산맥주,6
...,...,...,...
1637593,M999962961,호두,1
1637594,M999962961,호주산소우둔,1
1637595,M999962961,혼합견과,1
1637596,M999962961,혼합잡곡,1


In [20]:
df.columns = ['cust','소분류', '수량']


### 2. 코사인 함수 사용
#### - pivot table 변환
#### - svd 특이값 분해
#### - 코사인 함수 사용

In [21]:
# 데이터 프레임 컬럼명 정의
df.columns = ['cust','소분류', '수량']

# 피벗 테이블로 변환
df_pivot=df.pivot('cust','소분류','수량')

# NAN 값 0으로 변경
df_pivot4user=df_pivot.fillna(0)

# 위의 피벗테이블 넘파이 행렬로 변환
matrix = df_pivot4user.to_numpy()

# 타입변경
matrix = matrix.astype(int)

# svd 함수로 특이값 분해 (각 U, S, V)
U, S, V = randomized_svd(matrix, n_components=2)
S = np.diag(S)

# 분해한 값 합친후 원 데이터 형식으로 변경
svd_matrix = np.matmul(np.matmul(U,S), V)

# 코사인 유사도 활용 고객간 유사도 계산
cos_user = cosine_similarity(svd_matrix, svd_matrix)

# 위에서 구한 유사도 데이터 프레임으로 변경
df_cos = pd.DataFrame(data = cos_user, index=df_pivot4user.index, columns=df_pivot4user.index)


In [22]:
df = df.set_index('cust')

In [23]:
df_cos

cust,M000034966,M000136117,M000201112,M000225114,M000261625,M000350564,M000419293,M000494848,M000504230,M000508243,...,M999340261,M999441814,M999468256,M999492154,M999515910,M999599111,M999673157,M999770689,M999849895,M999962961
cust,,,,,,,,,,,,,,,,,,,,,
M000034966,1.000000,0.999261,0.997779,0.972310,0.998377,0.996094,0.999506,0.999587,0.998451,0.998158,...,0.998389,0.997448,0.997190,0.997576,0.999354,0.999901,0.996480,0.990554,0.974274,0.998401
M000136117,0.999261,1.000000,0.994481,0.962606,0.999828,0.998752,0.999975,0.997742,0.995574,0.999753,...,0.999832,0.993966,0.999333,0.999514,0.997233,0.999702,0.992521,0.984549,0.964889,0.999836
M000201112,0.997779,0.994481,1.000000,0.985716,0.992367,0.988000,0.995193,0.999282,0.999940,0.991901,...,0.992393,0.999988,0.989986,0.990726,0.999529,0.996745,0.999851,0.997487,0.987121,0.992418
M000225114,0.972310,0.962606,0.985716,1.000000,0.957423,0.947875,0.964483,0.978627,0.983805,0.956342,...,0.957484,0.986513,0.952072,0.953690,0.980080,0.968930,0.988478,0.995171,0.999963,0.957542
M000261625,0.998377,0.999828,0.992367,0.957423,1.000000,0.999506,0.999674,0.996327,0.993662,0.999993,...,1.000000,0.991764,0.999838,0.999920,0.995685,0.999079,0.990089,0.981137,0.959858,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999599111,0.999901,0.999702,0.996745,0.968930,0.999079,0.997236,0.999849,0.999084,0.997571,0.998912,...,0.999088,0.996347,0.998144,0.998455,0.998750,1.000000,0.995204,0.988529,0.971011,0.999096
M999673157,0.996480,0.992521,0.999851,0.988478,0.990089,0.985185,0.993353,0.998479,0.999601,0.989560,...,0.990119,0.999922,0.987401,0.988231,0.998849,0.995204,1.000000,0.998562,0.989737,0.990147
M999770689,0.990554,0.984549,0.997487,0.995171,0.981137,0.974575,0.985754,0.994087,0.996648,0.980410,...,0.981177,0.997816,0.977499,0.978610,0.994842,0.988529,0.998562,1.000000,0.995974,0.981217


In [24]:
data_df = pd.DataFrame(svd_matrix)

In [25]:
data_df

,0,1,2,3,4,5,6,7,8,9,...,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931
0,0.000341,0.001348,0.014325,0.000129,0.000290,0.007771,0.000056,0.000014,0.000731,0.006748,...,0.002805,0.002002,0.027980,0.032168,0.003527,0.002258,0.010639,0.003937,0.000129,0.000048
1,0.000118,0.000466,0.004965,0.000045,0.000101,0.002687,0.000019,0.000005,0.000253,0.002343,...,0.000974,0.000696,0.009690,0.011180,0.001219,0.000783,0.003701,0.001367,0.000045,0.000017
2,0.000252,0.000999,0.010569,0.000094,0.000213,0.005756,0.000041,0.000010,0.000541,0.004963,...,0.002064,0.001471,0.020676,0.023622,0.002615,0.001664,0.007798,0.002895,0.000095,0.000035
3,0.000726,0.002898,0.030304,0.000267,0.000598,0.016677,0.000119,0.000029,0.001564,0.014104,...,0.005869,0.004169,0.059518,0.066865,0.007595,0.004755,0.021964,0.008223,0.000277,0.000101
4,0.000082,0.000322,0.003438,0.000031,0.000070,0.001859,0.000013,0.000003,0.000175,0.001624,...,0.000675,0.000482,0.006706,0.007751,0.000843,0.000543,0.002567,0.000948,0.000031,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26912,0.000263,0.001038,0.011039,0.000099,0.000224,0.005983,0.000043,0.000011,0.000563,0.005204,...,0.002163,0.001545,0.021554,0.024813,0.002715,0.001741,0.008210,0.003036,0.000099,0.000037
26913,0.000118,0.000468,0.004951,0.000044,0.000099,0.002699,0.000019,0.000005,0.000254,0.002323,...,0.000966,0.000688,0.009688,0.011050,0.001227,0.000779,0.003646,0.001355,0.000045,0.000017
26914,0.001573,0.006251,0.065822,0.000585,0.001314,0.036001,0.000257,0.000064,0.003381,0.030795,...,0.012809,0.009118,0.128971,0.146341,0.016372,0.010348,0.048214,0.017960,0.000597,0.000220
26915,0.000235,0.000936,0.009798,0.000086,0.000194,0.005389,0.000038,0.000009,0.000505,0.004562,...,0.001898,0.001349,0.019240,0.021634,0.002454,0.001538,0.007108,0.002660,0.000089,0.000033


### 3. 피벗테이블 변환된 테이블

In [26]:
df_pivot

소분류,2단우산,3단우산,BB/파운데이션/컴팩트류,CDP/카세트,DIY가구,DIY완구,DSLR,DVD플레이어,LED,LED램프,...,후크류,훈련,훈제/구이오징어,훈제/로스트,휴대폰,휴대폰용품,휴지통,흑미,히터,힙색/사이드백
cust,,,,,,,,,,,,,,,,,,,,,
M000034966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M000136117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M000201112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M000225114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M000261625,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999599111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M999673157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M999770689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4. 결측값 0으로 처리

In [27]:
df_pivot4user

소분류,2단우산,3단우산,BB/파운데이션/컴팩트류,CDP/카세트,DIY가구,DIY완구,DSLR,DVD플레이어,LED,LED램프,...,후크류,훈련,훈제/구이오징어,훈제/로스트,휴대폰,휴대폰용품,휴지통,흑미,히터,힙색/사이드백
cust,,,,,,,,,,,,,,,,,,,,,
M000034966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M000136117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M000201112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M000225114,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M000261625,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999599111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M999673157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M999770689,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 5. 함수생성 -2
### - 유사도가 높은 고객 5명 추출

In [28]:
# user_Id 입력시 유사도가 가장높은 사람 5명 반환

def sim_user(user_id):
    return df_cos[user_id].sort_values(ascending=False)[1:6].index



sim_user('M785544658')

Index(['M912267161', 'M413492253', 'M830835354', 'M242749845', 'M071273003'], dtype='object', name='cust')

## 6. 함수생성 -3
### - 유사도가 높은 고객 5명이 가장 많이 산 상품 3개씩 추천 (중복제거)

In [29]:
# 위 함수에서 받은 유저가 가장 많이 산 제품 3개 추천 (중복 제거)

def recomendation(user_id):
    cust = most_sim(user_id)
    items = []
    for i in sim_user(cust):
        for j in range(3):
            if type(df.loc[i]) == pd.core.series.Series:
                pass
            else:
                try:
                    item = df.loc[i].sort_values('수량', ascending=False)[:3]['소분류'][j]
                    items.append(item)
                except:
                    pass
                
    recommendation_items = list(set(items))
    df_recommendation = pd.DataFrame(columns=['추천상품'])
    
    for i in range(len(recommendation_items)):
        df_recommendation.loc[i]=recommendation_items[i]
        
    return df_recommendation

## 7. 함수생성-3
### - 추천 받은 고객이 실제 구매한 상품 

In [30]:
def check(user_id):
    try:
        check_df = df.loc[user_id].sort_values('수량', ascending=False)
        check_df.columns=['구매상품', '수량']
    except:
        check_df = df[df.index == user_id]
        check_df.columns=['구매상품', '수량']
    return check_df

In [31]:
check('M223118731')[:4]

,구매상품,수량
cust,,
M223118731,물티슈,6
M223118731,관상어용품,2
M223118731,냉장치킨족발류,2
M223118731,샴푸,2


In [32]:
check('M499847718')[:4]

,구매상품,수량
cust,,
M499847718,베이커리,15
M499847718,갈치,3
M499847718,명태,2
M499847718,크래커,2


In [33]:
check('M062096421')[:4]

,구매상품,수량
cust,,
M062096421,남성티셔츠,14
M062096421,기타파티/팬시용품,6
M062096421,비빔봉지라면,3
M062096421,빵/케이크,2


In [34]:
check('M961351543')[:4]

,구매상품,수량
cust,,
M961351543,커피/음료,1


In [35]:
display(recomendation('M999770689'), check('M785544658')[:20])

,추천상품
0,기타생활서비스
1,임대매출
2,식빵
3,일반차음료
4,혼합탄산
5,수입담배
6,기타남성의류세트
7,얼음
8,여성남방셔츠
9,일반시리얼


,구매상품,수량
cust,,
M785544658,가공우유,40
M785544658,삼각김밥,21
M785544658,젤리,19
M785544658,커피음료,16
M785544658,옥수수스낵,14
M785544658,일반스낵,14
M785544658,비타민/에너지음료,12
M785544658,마시는요구르트,10
M785544658,빵/케이크,9
